# This jupyter is for merge all IRA dataset, to extract mentions and hashtags for each tweet and clean the mentions from special characters, moreover a division in various csv for each lang of troll is produced. The troll to troll network is produced and put in gephi format

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
pd.set_option('display.max_rows', 501)

## Create Main dataset concatenating all IRA dataset
This will create a dataset of over 3 million rows 


In [3]:
#concatenate all IRAhandle_tweets_*.csv files into one dataframe
df = pd.concat([pd.read_csv(f) for f in glob.glob('IRAhandle_tweets_*.csv')], ignore_index=True)


C:\Users\miro\AppData\Local\Temp/ipykernel_22236/610479456.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in glob.glob('IRAhandle_tweets_*.csv')], ignore_index=True)
C:\Users\miro\AppData\Local\Temp/ipykernel_22236/610479456.py:2: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in glob.glob('IRAhandle_tweets_*.csv')], ignore_index=True)
C:\Users\miro\AppData\Local\Temp/ipykernel_22236/610479456.py:2: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in glob.glob('IRAhandle_tweets_*.csv')], ignore_index=True)


Read merged dataset

In [36]:
df=pd.read_csv("complete_538.csv")

C:\Users\miro\AppData\Local\Temp/ipykernel_22924/622885350.py:1: DtypeWarning: Columns (1,11,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("complete_538.csv")


In [37]:
#count na values in each column
col=["Unnamed: 0","external_author_id","tco1_step1", "harvested_date","following","updates","post_type","account_type","new_june_2018","alt_external_id","tweet_id","article_url","tco2_step1","tco3_step1"]
#drop unnecessary columns in col
df.drop(col,axis=1,inplace=True)

In [38]:
#extract mentions
import re
mentions=df.copy()
mentions['mentions'] = mentions['content'].str.findall(r'[＠@]([^][\s#<>|{}]+)')



In [40]:
#assign to each account it's max value of followers reached for future network visualization
mentions['followers'] = mentions.groupby('author')['followers'].transform('max')
#drop rows wihout mentions
mentions=mentions[mentions.astype(str)['mentions'] != '[]']

In [ ]:
#using mentions dataset just created and use it to create a parallel hashtag dataset
hashtags=mentions.copy()
hashtags["hashtags"] = hashtags["content"].str.findall(r'[#]([^][\s#<>|{}]+)')


In [44]:
#drop empty hashtags list
hashtags=hashtags[df.astype(str)['hashtags'] != '[]']
#create n rows for n hashtags used in tweet
hashtags=hashtags.explode("hashtags",ignore_index=True)



In [47]:
#create n rows for n account mentioned
mentions=mentions.explode("mentions",ignore_index=True)
hashtags=hashtags.explode("mentions",ignore_index=True)
df1=hashtags.copy()

In [48]:
#remove special characters from mentions
mentions['mentions'] = mentions['mentions'].str.replace('[^\w\s]','')
hashtags['mentions'] = hashtags['mentions'].str.replace('[^\w\s]','')


C:\Users\miro\AppData\Local\Temp/ipykernel_22924/2461692558.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['mentions'] = df1['mentions'].str.replace('[^\w\s]','')


In [49]:
#convert mentions to uppercase beacause author is upper case
mentions['mentions'] = mentions['mentions'].str.upper()
hashtags['mentions'] = hashtags['mentions'].str.upper()



In [50]:
#extract year for each entry in publish date dd/mm/yyyy
hashtags['year'] = hashtags['publish_date'].str.split('/').str.get(2)
#convert year to datetime
hashtags['year'] = pd.to_datetime(hashtags['year'])
#extract year from column year
hashtags['year'] = hashtags['year'].dt.year


In [ ]:
#extract year for each entry in publish date dd/mm/yyyy
mentions['year'] = mentions['publish_date'].str.split('/').str.get(2)
#convert year to datetime
mentions['year'] = pd.to_datetime(mentions['year'])
#extract year from column year
mentions['year'] = mentions['year'].dt.year

In [141]:
#filtering for english language and italian language
english_hash=hashtags[hashtags['language'].str.contains('English')]
english=mentions[mentions['language'].str.contains('English')]
italian=mentions[mentions['language'].str.contains('Italian')]

In [59]:
#clean special characters from hashtags
english_hash['hashtags'] = english_hash['hashtags'].str.replace("'", "")
english_hash['hashtags'] = english_hash['hashtags'].str.replace("[^\w\s]", "")
english_hash['hashtags'] = english_hash['hashtags'].str.replace("_", "")
english_hash['hashtags'] = english_hash['hashtags'].str.replace("-", "")
english_hash['hashtags'] = english_hash['hashtags'].str.replace(".", "")
english_hash['hashtags'] = english_hash['hashtags'].str.replace("/", "")



C:\Users\miro\AppData\Local\Temp/ipykernel_22924/3257834028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_hash['hashtags'] = english_hash['hashtags'].str.replace("'", "")
C:\Users\miro\AppData\Local\Temp/ipykernel_22924/3257834028.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  english_hash['hashtags'] = english_hash['hashtags'].str.replace("[^\w\s]", "")
C:\Users\miro\AppData\Local\Temp/ipykernel_22924/3257834028.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [33]:
#filtering for russian language
russian=mentions[mentions['language'].str.contains('Russian')]
russian.to_csv("russian_lang_troll.csv")

In [142]:
#exporting datasets
russian.to_csv("russian_lang_troll.csv")
english.to_csv("english_lang_troll.csv")
english_hash.to_csv("english_lang_troll_hash.csv")
italian.to_csv("italian_lang_troll.csv")
df.to_csv("all_lang_troll.csv")

So we created two types of dataset one focused on mentions named "mentions" and another focused on hashtags with hashtags and mentions exploded

## CREATE TROLL TO TROLL NETWORK

In [280]:
#convert author to lower string
trollo=df.copy()
#keep elements only if element in author column are present in mention column


In [281]:
#check if mentions correspond to the author, to create only a network of trolls
trollo["exist_target"]=trollo.mentions.isin(trollo.author)

In [284]:
trollo["exist_target"].value_counts()


False    1153605
True       11071
Name: exist_target, dtype: int64

In [285]:
#keep only exist_target==True
troll_troll=trollo[trollo['exist_target']==True]

In [1]:
troll_troll.to_csv("troll_troll_all_lang.csv")

NameError: name 'troll_troll' is not defined

In [18]:
troll_troll=pd.read_csv("troll_troll_all_lang.csv")

 # Load Network on Networkx and prepare data for Gephi
 First we create the edge list for Gephi

In [20]:
import networkx as nx

In [21]:
G2 = nx.from_pandas_edgelist(troll_troll, source='author', target='mentions',
                                     edge_attr=['followers', 'account_category', 'language', 'region'],
                                        create_using=nx.DiGraph())
                                        

In [24]:
#select node attributes we are interested in
src_attributes = ['followers', 'account_category', 'publish_date']

# Iterate over df rows and set source node attributes:
for index, row in troll_troll.iterrows():
    src_attr_dict = {k: row.to_dict()[k] for k in src_attributes}    
    G2.nodes[row['author']].update(src_attr_dict)

In [26]:
#write graphml file for gephu
nx.write_graphml(G2,"troll_troll1.graphml")

In [25]:
#show node attribute
G2.nodes.data()

NodeDataView({'10_GOP': {'followers': 10465, 'account_category': 'RightTroll', 'publish_date': '9/8/2017 2:59'}, 'TEN_GOP': {'followers': 145244, 'account_category': 'RightTroll', 'publish_date': '9/25/2016 0:58'}, 'ELEVEN_GOP': {'followers': 59100, 'account_category': 'RightTroll', 'publish_date': '7/5/2017 3:33'}, 'REALTEN_GOP': {'followers': 19754, 'account_category': 'RightTroll', 'publish_date': '8/28/2016 2:34'}, '1D_NICOLE_': {'followers': 53, 'account_category': 'Fearmonger', 'publish_date': '8/12/2015 8:02'}, 'DRGREGORYWEST': {'followers': 116, 'account_category': 'RightTroll', 'publish_date': '8/11/2015 17:43'}, '4EVER1937': {'followers': 11960, 'account_category': 'NonEnglish', 'publish_date': '5/25/2017 8:34'}, '4MYSQUAD': {'followers': 4811, 'account_category': 'LeftTroll', 'publish_date': '9/27/2015 13:05'}, 'GALLOWAYELIANA': {'followers': 1158, 'account_category': 'LeftTroll', 'publish_date': '9/25/2015 3:32'}, 'JOHNNYMARCH_': {'followers': 816, 'account_category': 'Left